# Use Case 2: New Zealand Traffic Crash Analysis
Road safety analytics requires processing extensive crash records to identify patterns and risk factors. Using Waka Kotahi's Crash Analysis System (CAS), we'll visualise traffic crash distributions across New Zealand's road network, revealing spatial clustering, temporal trends, and contributing factors that inform evidence-based road safety interventions.

## Key Technical Highlights

- **Dataset Scale**: 25+ years of comprehensive crash records across New Zealand
- **Performance**: GPU-accelerated rendering of crash density patterns and hotspots
- **Interactivity**: Real-time filtering by crash severity, conditions, and contributing factors
- **Memory Efficiency**: PyArrow's efficient handling of categorical crash variables and temporal data

## What You'll See
This demonstration showcases how traffic crashes distribute across New Zealand's diverse road environments, identifying high-risk corridors, seasonal patterns, and contributing factors that provide actionable insights for targeted road safety improvements through high-performance geospatial visualisation.

## Datasets
### Traffic Crash Records: Waka Kotahi CAS
Our crash data comes from Waka Kotahi's Crash Analysis System (CAS) - New Zealand's authoritative road crash database:

<img src="images/cas.gif" alt="drawing" width="1200"/>

[https://opendata-nzta.opendata.arcgis.com/datasets/NZTA::crash-analysis-system-cas-data-1/about](https://opendata-nzta.opendata.arcgis.com/datasets/NZTA::crash-analysis-system-cas-data-1/about)

- Coverage: All traffic crashes reported by NZ Police since 1 January 2000
- Scope: Crashes on all New Zealand roadways and publicly accessible areas
- Data Currency: Monthly updates in the first week of each month
- Privacy Protection: Non-personal crash variables only, ensuring individual anonymity

The CAS database captures comprehensive crash characteristics, including:

- Temporal Variables: Date, time, and day of week
- Spatial Variables: Location coordinates and road classification
- Severity Classifications: Fatal, serious injury, minor injury, and non-injury crashes
- Contributing Factors: Weather conditions, road surface, speed limits, and crash causes
- Vehicle Details: Vehicle types, movements, and involvement patterns

In [1]:
from numpy import column_stack
from ipywidgets import FloatRangeSlider, jsdlink
from lonboard import Map, ScatterplotLayer
from lonboard.controls import MultiRangeSlider
from lonboard.layer_extension import DataFilterExtension
from pyarrow.compute import fill_null, max as pc_max, min as pc_min
from pyarrow.parquet import read_table, write_table
from arro3.io import read_parquet

In [2]:
# Get data
crashes = read_table("data/crashes-pre-05-11-2025.parquet")

In [3]:
# Set how we want to be able to filter the map
filter_columns = ["crashYear", "fatalCount", "seriousInjuryCount", "minorInjuryCount"]
filter_extension = DataFilterExtension(filter_size=len(filter_columns))

filter_values = column_stack([fill_null(crashes[column], 0) for column in filter_columns])
initial_filter_range = [
    [float(pc_min(crashes[column])), float(pc_max(crashes[column]))]
    for column in filter_columns
]

In [4]:
# Create a layer
layer = ScatterplotLayer(
    crashes,
    extensions=[filter_extension],
    get_fill_color=[11, 41, 72],
    radius_min_pixels=2,
    radius_max_pixels=10,
    get_filter_value=filter_values,
    filter_range=initial_filter_range,
    pickable=True,
)

In [8]:
# And add the layer to a map
m = Map(
    layer,
    show_tooltip=True,
    show_side_panel=True,
    view_state={"latitude": -41.286062, "longitude": 172.760010, "zoom": 4.1},
)
m

In [6]:
year_slider = FloatRangeSlider(
    value=initial_filter_range[0],
    min=initial_filter_range[0][0],
    max=initial_filter_range[0][1],
    step=1,
    description="Year: ",
)
fatal_slider = FloatRangeSlider(
    value=initial_filter_range[1],
    min=initial_filter_range[1][0],
    max=initial_filter_range[1][1],
    step=1,
    description="Fatal count: ",
)
serious_injury_slider = FloatRangeSlider(
    value=initial_filter_range[2],
    min=initial_filter_range[2][0],
    max=initial_filter_range[2][1],
    step=1,
    description="Serious injury count: ",
)
minor_injury_slider = FloatRangeSlider(
    value=initial_filter_range[2],
    min=initial_filter_range[3][0],
    max=initial_filter_range[3][1],
    step=1,
    description="Minor injury count: ",
)
multi_slider = MultiRangeSlider(
    [year_slider, fatal_slider, serious_injury_slider, minor_injury_slider]
)
multi_slider

MultiRangeSlider(children=(FloatRangeSlider(value=(2000.0, 2025.0), description='Year: ', max=2025.0, min=2000…

In [7]:
_ = jsdlink((multi_slider, "value"), (layer, "filter_range"))